In [ ]:
import psycopg2
import psycopg2.extras
import time
from datetime import datetime
import sys
import subprocess
import requests
import json

In [ ]:
DB_HOST='localhost'
POSTGRES_DB='pg_dev'
POSTGRES_USER='openbot'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

In [ ]:
#Rotina principal

max_requests = 1000 #(provisório)

while True:
    
    #Busca pela tabela de pares
    pairs = db_get_pairs_table
    
    if (len(pairs()) > 0):
        
        print('sucesso')
            
        for pair in pairs():
            
            if (pair['status'] == 'ON'):
                
                candles_table_name = 'klines_1m_' + str(pair['pair_name'])
                pointer_action = 'SELECT * FROM ' + candles_table_name

                #Verifica se há a tabela de candles:
                try:
                    
                    connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                                  user=POSTGRES_USER, password=POSTGRES_PASSWORD)
                    
                    pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

                    pointer.execute(pointer_action)

                    #Adquire os dados históricos, caso não existam:
                    if (pair['historical'] == 'absent'):

                        #Historical update
                        pair = str(pair['pair_name'])

                        #pid = subprocess.Popen([sys.executable, "teste.py", pair, max_requests])

                        pid = get_historical_data(pair, max_requests)

                        print('PID is ', pid)

                    elif (pair['historical'] == 'full'):
                        
                        print('massa de dados completa')

                        candles = pointer.fetchall()
                        #latest_candle = candles[len(candles)]

                        #Current update
                
                #Se não existe, a cria:
                except (Exception, psycopg2.Error) as error:
                    
                    print('tabela inexistente, criando-a')
                    print(' ')

                    print(error)

                    pointer.execute("ROLLBACK")

                    pointer_action = "CREATE TABLE " + candles_table_name + ''' 
                    (open_time timestamp PRIMARY KEY, open numeric, high numeric, low numeric, close numeric,
                    volume numeric, quote_asset_volume numeric, number_of_trades integer, 
                    taker_buy_base_asset_volume numeric, taker_buy_quote_asset_volume numeric);
                    '''
                    pointer.execute(pointer_action)
                    
                finally:
                    
                    if (connection):
                        pointer.close()
                        connection.close()
                    
        time.sleep(60) #Ajeitar para acordar poucos segundos após fechamento do próximo candle
    
    #Descansa 5min se não houver a tabela de pares
    else:
        
        print('pairs ausente')
        time.sleep(300)

In [ ]:
def db_get_pairs_table():
    
    pairs = []
    
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

        sql_select_query = 'SELECT * FROM getdata_pair'

        pointer.execute(sql_select_query)
        
        pairs = pointer.fetchall()
        
        #return pairs
        
    except (Exception, psycopg2.Error) as error:

        #print("Error in update operation", error)

        pointer.execute("ROLLBACK")

    finally:

        if (connection):
            pointer.close()
            connection.close()
    
    return pairs
    
def db_update_pair_table(pair, historical):
    
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

        sql_update_query = """Update getdata_pair set historical = %s where pair_name = %s"""

        pointer.execute(sql_update_query, (historical, pair))

        connection.commit()

        return 'done'

    except (Exception, psycopg2.Error) as error:

        print("Error in update operation", error)

        pointer.execute("ROLLBACK")

        return 'fail'

    finally:

        if (connection):
            pointer.close()
            connection.close()
            
def db_save_kline(pair,klines):
    
    candles_table_name = 'klines_1m_' + pair
    
    SQL = '''
    INSERT INTO ''' + candles_table_name + '''
    (open_time, open, high, low, close, volume, 
    quote_asset_volume, number_of_trades, taker_buy_base_asset_volume, 
    taker_buy_quote_asset_volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    try:
        
        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)
        
        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
        for kline in klines:
        
            data = (datetime.fromtimestamp(kline[0]/1000), kline[1], kline[2], 
                    kline[3], kline[4], kline[5], kline[7], kline[8], kline[9], kline[10],)
        
            pointer.execute(SQL, data)
        
        return 'done'
        
    except (Exception, psycopg2.Error) as error:
        
        print("Error in update operation", error)
        
        pointer.execute("ROLLBACK")
        
        return 'fail'
        
    finally:

        if (connection):
            
            connection.commit()
            pointer.close()
            connection.close()

In [ ]:
def get_historical_data(pair, max_requests):
    
    db_update_status = db_update_pair_table(pair,'building')

    num_requests = 0
    
    for i in range(10):
        
        url = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime=1241893500000'
        
        try:

            oldest_klines = requests.get(url)
            
            num_requests+= 1
            
            break
            
        except Exception as error:
            
            print('Comunicação com a binance falhou na entrada, erro: ',error) #Na verdade enviar para destino apropriado

            time.sleep(5)
        
    if (oldest_klines):
        
        db_save_kline_status = db_save_kline(pair,oldest_klines)
        
        if (db_save_kline_status == 'done'):
            
            step = 60000
            delta_time = 500*step
            start_time = int((oldest_klines.json()[0][0])) + delta_time
            ciclo = 0
            
            while True:
                
                for i in range(10):
                    
                    url = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(start_time)
                    
                    try:
                        
                        new_klines = requests.get(url)
                        
                        num_requests+= 1
                        
                        break

                    except Exception as error:
                        
                        print('Comunicação com a binance falhou no cliclo ',ciclo,', tentativa ',i,'. Erro: ', error) #Na verdade enviar para destino apropriado

                        time.sleep(5)

                if (new_klines):
                    
                    if (len(new_klines.json()) < 500): #Chegou no mais recente
                        
                        print('dado mais atual, este foi o último ciclo')
                        
                        new_klines = new_klines[:(len(new_klines)-1)]
                        
                    db_save_kline_status = db_save_kline(pair,new_klines)
                    
                    #if (db_save_kline_status == 'done'):
                    
                    start_time = start_time + delta_time
                            
                    if (len(new_klines.json()) < 500): 

                        break

                    if (len(new_klines.json()) > 500): #Alguma anomalia.

                        print('ATENÇÃO, MAIS DE 500 AMOSTRAS')
                        break

                print ('Número de requests: ', num_requests)

                if(num_requests > int(max_requests)):

                    print('número máximo de requests atingido, dormindo')
                    num_requests = 0
                    time.sleep(60)

                ciclo+=1

            #Atualiza, na tabela "getdata_pairs", os 'historical' dos pares "absent", de "building" para "full"
            db_update_status = db_update_pair_table(pair,'full')
    
    pid = 23
    return pid

In [ ]:
pid = get_historical_data('BTCUSDT',1000)

NameError: name 'requests' is not defined